> MongoDB 是由 C++ 语言编写的非关系型数据库，是一个基于分布式文件存储的开源数据库系统，其内容存储形式类似 Json 对象，它的字段值可以包含其他文档，数组及文档数组，非常灵活。



## 1.连接MongoDB

In [3]:
import pymongo

# 创建 MongoDB 对象
client = pymongo.MongoClient('localhost', port=27017)
# client = pymongo.MongoClient('mongodb://localhost:27017/')

## 2.指定数据库

In [4]:
db = client.test    # test 指的是数据库名
# db = client['test']

## 3.指定集合（`collection`）

- `集合collection`类似关系型数据库中的`表table`

In [55]:
collection = db.students
# collection = db['students']

## 4.插入数据

In [11]:
![图 1](../../../images/f881119a8bb859cfefc9e0d94b7a155c5ac9c1e435485aa251f930cb53787098.png)  
student = {
    'id':'20170101',
    'name':'Jordan',
    'age':20,
    'gender':'male'
}

result = collection.insert_one(student)
# collection.insert_one()   执行后返回  InsertOneResult 对象,包含inserted_id属性
# collection.insert()
# 注意区别，现在不再使用insert()方法了

print(result.inserted_id)

62a9824cc2bfa89e5422fb83


![图 2](../../../images/f881119a8bb859cfefc9e0d94b7a155c5ac9c1e435485aa251f930cb53787098.png)  


In [17]:
student1 = {
    'id':'20170101',
    'name':'Jordan',
    'age':20,
    'gender':'male'
}

student2 = {
    'id':'20170202',
    'name':'Mike',
    'age':21,
    'gender':'male'
}

result = collection.insert([student1, student2])
print(result)

TypeError: 'Collection' object is not callable. If you meant to call the 'inserts' method on a 'Collection' object it is failing because no such method exists.

```
TypeError: 'Collection' object is not callable. If you meant to call the 'inserts' method on a 'Collection' object it is failing because no such method exists.
```

- 官方文档说明
[collection insert is removed](https://pymongo.readthedocs.io/en/stable/migrate-to-pymongo4.html#collection-insert-is-removed)

In [29]:
student1 = {
    'id':'20170101',
    'name':'Jordan',
    'age':20,
    'gender':'male'
}

student2 = {
    'id':'20170202',
    'name':'Mike',
    'age':21,
    'gender':'male'
}

result = collection.insert_many([student1, student2])
print(result)
print(result.inserted_ids)

[ObjectId('62a98536c2bfa89e5422fb8c'), ObjectId('62a98536c2bfa89e5422fb8d')]


## 5.查询

In [31]:
res = collection.find_one({'name':'Mike'})
print(type(res))
print(res)

<class 'dict'>
{'_id': ObjectId('62a983d4c2bfa89e5422fb85'), 'id': '20170202', 'name': 'Mike', 'age': 21, 'gender': 'male'}


- 返回第一个结果
![图 3](../../../images/f78baa23496b87f2a2af1e7621ffd4f8b94acb88e58e042435b38550a39b7623.png)  


- 根据 `ObjectId` 查询

In [32]:
from bson.objectid import ObjectId

res = collection.find_one({'_id':ObjectId('62a983d4c2bfa89e5422fb85')})
print(res)

{'_id': ObjectId('62a983d4c2bfa89e5422fb85'), 'id': '20170202', 'name': 'Mike', 'age': 21, 'gender': 'male'}


>多条数据查询

- `col.find()`

返回结果是 `Cursor` 类型

In [36]:
res = collection.find({'name':'Mike'})
print(type(res))
for i in res:
    print(i)

<class 'pymongo.cursor.Cursor'>
{'_id': ObjectId('62a983d4c2bfa89e5422fb85'), 'id': '20170202', 'name': 'Mike', 'age': 21, 'gender': 'male'}
{'_id': ObjectId('62a98515c2bfa89e5422fb87'), 'id': '20170202', 'name': 'Mike', 'age': 21, 'gender': 'male'}
{'_id': ObjectId('62a9851dc2bfa89e5422fb89'), 'id': '20170202', 'name': 'Mike', 'age': 21, 'gender': 'male'}
{'_id': ObjectId('62a98531c2bfa89e5422fb8b'), 'id': '20170202', 'name': 'Mike', 'age': 21, 'gender': 'male'}
{'_id': ObjectId('62a98536c2bfa89e5422fb8d'), 'id': '20170202', 'name': 'Mike', 'age': 21, 'gender': 'male'}


> 查询年龄大于20的数据

In [38]:
res = collection.find({'age':{'$gt':20}})
for i in res:
    print(i)

{'_id': ObjectId('62a983d4c2bfa89e5422fb85'), 'id': '20170202', 'name': 'Mike', 'age': 21, 'gender': 'male'}
{'_id': ObjectId('62a98515c2bfa89e5422fb87'), 'id': '20170202', 'name': 'Mike', 'age': 21, 'gender': 'male'}
{'_id': ObjectId('62a9851dc2bfa89e5422fb89'), 'id': '20170202', 'name': 'Mike', 'age': 21, 'gender': 'male'}
{'_id': ObjectId('62a98531c2bfa89e5422fb8b'), 'id': '20170202', 'name': 'Mike', 'age': 21, 'gender': 'male'}
{'_id': ObjectId('62a98536c2bfa89e5422fb8d'), 'id': '20170202', 'name': 'Mike', 'age': 21, 'gender': 'male'}


|符号|含义|示例|
|:-:|:-:|:-:|
|`$lt`|小于|`{'age':{'$lt:20'}}`|
|`$gt`|大于|`{age':{'$gt':19}}`|
|`$ne`|不等于|``|
|`$in`|在范围内|`{'age':{'$in':[20, 23]}}`|
|`$nin`|不再范围内|``|
|`$lte`|小于等于|``|
|`$gte`|大于等于|``|


> 查询名字以 `M` 开头的学生数据

In [49]:
res = collection.find({'name':{'$regex':'^M.*'}})
[i for i in res]

[{'_id': ObjectId('62a983d4c2bfa89e5422fb85'),
  'id': '20170202',
  'name': 'Mike',
  'age': 21,
  'gender': 'male'},
 {'_id': ObjectId('62a98515c2bfa89e5422fb87'),
  'id': '20170202',
  'name': 'Mike',
  'age': 21,
  'gender': 'male'},
 {'_id': ObjectId('62a9851dc2bfa89e5422fb89'),
  'id': '20170202',
  'name': 'Mike',
  'age': 21,
  'gender': 'male'},
 {'_id': ObjectId('62a98531c2bfa89e5422fb8b'),
  'id': '20170202',
  'name': 'Mike',
  'age': 21,
  'gender': 'male'},
 {'_id': ObjectId('62a98536c2bfa89e5422fb8d'),
  'id': '20170202',
  'name': 'Mike',
  'age': 21,
  'gender': 'male'}]

![图 5](../../../images/ce0b9fe84e3d493d03199ec5ad936dd9889cf2d668fadbe6a71c5fd785a32bda.png)  


In [52]:
[i for i in collection.find({'name':{'$exists':True}})]

[{'_id': ObjectId('62a9815ec2bfa89e5422fb80'),
  'id': '20170101',
  'name': 'Jordan',
  'age': 20,
  'gender': 'male'},
 {'_id': ObjectId('62a98164c2bfa89e5422fb81'),
  'id': '20170101',
  'name': 'Jordan',
  'age': 20,
  'gender': 'male'},
 {'_id': ObjectId('62a98169c2bfa89e5422fb82'),
  'id': '20170101',
  'name': 'Jordan',
  'age': 20,
  'gender': 'male'},
 {'_id': ObjectId('62a9824cc2bfa89e5422fb83'),
  'id': '20170101',
  'name': 'Jordan',
  'age': 20,
  'gender': 'male'},
 {'_id': ObjectId('62a983d4c2bfa89e5422fb84'),
  'id': '20170101',
  'name': 'Jordan',
  'age': 20,
  'gender': 'male'},
 {'_id': ObjectId('62a983d4c2bfa89e5422fb85'),
  'id': '20170202',
  'name': 'Mike',
  'age': 21,
  'gender': 'male'},
 {'_id': ObjectId('62a98515c2bfa89e5422fb86'),
  'id': '20170101',
  'name': 'Jordan',
  'age': 20,
  'gender': 'male'},
 {'_id': ObjectId('62a98515c2bfa89e5422fb87'),
  'id': '20170202',
  'name': 'Mike',
  'age': 21,
  'gender': 'male'},
 {'_id': ObjectId('62a9851dc2bfa89e5

## 6.计数

In [57]:
count = collection.find({}).count()
count

AttributeError: 'Cursor' object has no attribute 'count'

```
AttributeError: 'Cursor' object has no attribute 'count'
```

- 官方文档说明

[Collection.count and Cursor.count is removed¶](https://pymongo.readthedocs.io/en/stable/migrate-to-pymongo4.html#collection-count-and-cursor-count-is-removed)


```Python

col.estimated_document_count()
col.count_documents({'price':{'$gte':10}})
```

In [64]:
count1 = collection.estimated_document_count()
count2 = collection.count_documents({'name':'Mike'})

print(count1)
print(count2)

14
5


## 7.排序

- `pymongo.ASCENDING`  升序

- `pymongo.DESCENDING` 降序

In [72]:
res = collection.find().sort('age', pymongo.ASCENDING)
print(res)
[(i['_id'], i['name'], i['age']) for i in res]

[(ObjectId('62a9815ec2bfa89e5422fb80'), 'Jordan', 20),
 (ObjectId('62a98164c2bfa89e5422fb81'), 'Jordan', 20),
 (ObjectId('62a98169c2bfa89e5422fb82'), 'Jordan', 20),
 (ObjectId('62a9824cc2bfa89e5422fb83'), 'Jordan', 20),
 (ObjectId('62a983d4c2bfa89e5422fb84'), 'Jordan', 20),
 (ObjectId('62a98515c2bfa89e5422fb86'), 'Jordan', 20),
 (ObjectId('62a9851dc2bfa89e5422fb88'), 'Jordan', 20),
 (ObjectId('62a98531c2bfa89e5422fb8a'), 'Jordan', 20),
 (ObjectId('62a98536c2bfa89e5422fb8c'), 'Jordan', 20),
 (ObjectId('62a983d4c2bfa89e5422fb85'), 'Mike', 21),
 (ObjectId('62a98515c2bfa89e5422fb87'), 'Mike', 21),
 (ObjectId('62a9851dc2bfa89e5422fb89'), 'Mike', 21),
 (ObjectId('62a98531c2bfa89e5422fb8b'), 'Mike', 21),
 (ObjectId('62a98536c2bfa89e5422fb8d'), 'Mike', 21)]

## 8.偏移

>利用 `skip` 方法偏移几个位置，比如偏移 2，就忽略前两个元素，得到第三个及以后的元素

In [76]:
res = collection.find().sort('age', pymongo.ASCENDING).skip(7)
print(res)
[(i['_id'], i['name'], i['age']) for i in res]

[(ObjectId('62a98531c2bfa89e5422fb8a'), 'Jordan', 20),
 (ObjectId('62a98536c2bfa89e5422fb8c'), 'Jordan', 20),
 (ObjectId('62a983d4c2bfa89e5422fb85'), 'Mike', 21),
 (ObjectId('62a98515c2bfa89e5422fb87'), 'Mike', 21),
 (ObjectId('62a9851dc2bfa89e5422fb89'), 'Mike', 21),
 (ObjectId('62a98531c2bfa89e5422fb8b'), 'Mike', 21),
 (ObjectId('62a98536c2bfa89e5422fb8d'), 'Mike', 21)]

> `limit` 方法指定要取的结果个数

In [78]:
res = collection.find().sort('age', pymongo.ASCENDING).skip(7).limit(3)
print(res)
[(i['_id'], i['name'], i['age']) for i in res]

[(ObjectId('62a9815ec2bfa89e5422fb80'), 'Jordan', 20),
 (ObjectId('62a98515c2bfa89e5422fb86'), 'Jordan', 20),
 (ObjectId('62a98515c2bfa89e5422fb87'), 'Mike', 21)]

> 值得注意的是，在数据库数量非常庞大的时候，如千万、亿级别，最好不要使用大的偏移量来查询数据，因为这样很可能导致内存溢出。

> 此时可以使用类似如下操作来查询

In [79]:
from bson.objectid import ObjectId
collection.find({'_id': {'$gt': ObjectId('593278c815c2602678bb2b8d')}})

## 9.更新

> `update`方法

In [92]:
res = collection.update_one({'name':'Kevin'}, {'$set':{'age':999}})

{'_id': ObjectId('62a9815ec2bfa89e5422fb80'), 'id': '20170101', 'name': 'Jordan', 'age': 999, 'gender': 'male'}


![图 6](../../../images/c51adbb19c2229c7004ab9bbd39a446c7439d5974ae3f3fa78336dd57fcef603.png)  


In [ ]:
res = collection.update_one({'name':'Jordan'}, {'$set':{'age':999}})

![图 9](../../../images/6e16ac80a09b73ff61284ae3fe1cf87c0d868bffbb6fae91351dbbe6928b0cde.png)  


In [97]:
res = collection.update_many({'name':'Mike'}, {'$set':{'age':111}})

![图 10](../../../images/b373f98bd9e05ba9b80a168f2b2b3cdd596d936bdb930eba577946f32ccffe85.png)  


In [99]:
condition = {'name': 'Kevin'}
student = collection.find_one(condition)
student['age'] = 55

'''
    需要注意, 此时:
    student = {'_id': ObjectId('62a98164c2bfa89e5422fb81'), 'id': '20170101', 'name': 'Kevin', 'age': 55, 'gender': 'male'}

'''

result = collection.update_one(condition, {'$set': student})

In [107]:
student = collection.find_one(condition)
student['age'] = 100
print(student)

print(collection.find_one({'name':'Kevin'}))

# 发现一个age是100另一个还是55

{'_id': ObjectId('62a98164c2bfa89e5422fb81'), 'id': '20170101', 'name': 'Kevin', 'age': 100, 'gender': 'male'}
{'_id': ObjectId('62a98164c2bfa89e5422fb81'), 'id': '20170101', 'name': 'Kevin', 'age': 55, 'gender': 'male'}


`update` 方法其实也是官方不推荐使用的方法。这里也分为 `update_one` 方法和 `update_many` 方法，用法更加严格，它们的第二个参数需要使用 `$` 类型操作符作为字典的键名

- `update_one()`

In [118]:
condition = {'age':{'$gt':20}}
res = collection.update_one(condition, {'$inc':{'age': 3}})
print(res)
print(res.matched_count, res.modified_count)
# 匹配的数据条数 和 影响的数据条数
[i for i in collection.find({'age':{'$gt':20}}).limit(1)]

1 1


[{'_id': ObjectId('62a9815ec2bfa89e5422fb80'),
  'id': '20170101',
  'name': 'Jordan',
  'age': 1017,
  'gender': 'male'}]

- `update_many()`

In [120]:
condition = {'age':{'$gt':20}}
res = collection.update_many(condition, {'$inc':{'age': 7}})
print(res)
print(res.matched_count, res.modified_count)
# 匹配的数据条数 和 影响的数据条数
for i in collection.find({'age':{'$gt':20}}):
    print(i)

7 7
{'_id': ObjectId('62a9815ec2bfa89e5422fb80'), 'id': '20170101', 'name': 'Jordan', 'age': 1031, 'gender': 'male'}
{'_id': ObjectId('62a98164c2bfa89e5422fb81'), 'id': '20170101', 'name': 'Kevin', 'age': 62, 'gender': 'male'}
{'_id': ObjectId('62a983d4c2bfa89e5422fb85'), 'id': '20170202', 'name': 'Mike', 'age': 118, 'gender': 'male'}
{'_id': ObjectId('62a98515c2bfa89e5422fb87'), 'id': '20170202', 'name': 'Mike', 'age': 118, 'gender': 'male'}
{'_id': ObjectId('62a9851dc2bfa89e5422fb89'), 'id': '20170202', 'name': 'Mike', 'age': 118, 'gender': 'male'}
{'_id': ObjectId('62a98531c2bfa89e5422fb8b'), 'id': '20170202', 'name': 'Mike', 'age': 118, 'gender': 'male'}
{'_id': ObjectId('62a98536c2bfa89e5422fb8d'), 'id': '20170202', 'name': 'Mike', 'age': 118, 'gender': 'male'}
